In [8]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

import itk

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [9]:
num_mask = 3
im_mask = []
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth5_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth12_Edited.png", itk.F)))
im_mask.append(itk.GetArrayFromImage(itk.imread("EstimatedMask_Depth16_Edited.png", itk.F)))
im_mask_depth = [5,12,16]

In [10]:
def convert_video_to_linear(pathname):
    data_paths = sorted(glob(pathname+"/*.m??"))
    for i in range(len(data_paths)):
        vid = load_video(data_paths[i])
        print(i, data_paths[i])
        depth,zoom,offsetX,offsetY = get_depth_and_zoom_C52(vid[0])
        resampled_vid = linearize_video(vid)
        print(depth, zoom)
        itkimg = itk.GetImageFromArray(resampled_vid.astype(np.float32))
        filename = pathname+"-Linear/"+os.path.basename(data_paths[i])[:-4]+".mha"
        itk.imwrite(itkimg, filename)
        
def convert_annotations_to_linear(pathname):
    data_paths = sorted(glob(pathname+"-Annotations/*f?????.mha"))
    for i in range(len(data_paths)):
        ovfile = pathname+"-Annotations/"+os.path.basename(data_paths[i])[:-4]+".overlay.mha"
        itkov = itk.imread(ovfile)
        itkimg = itk.imread(data_paths[i])
        depth,zoom,offsetX,offsetY = get_depth_and_zoom_C52(itkimg)
        resampled_ov = linearize_image(itkov,depth,zoom,offsetY,False)  #use False for interpolate when converting overlays
        print(depth, zoom)
        itkresampled_ov = itk.GetImageFromArray(resampled_ov.astype(np.float32))
        filename = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])[:-4]+".overlay.mha"
        itk.imwrite(itkresampled_ov, filename)
        
def convert_slices_to_linear(pathname):
    data_paths = sorted(glob(pathname+"-Annotations/*f?????.mha"))
    for i in range(len(data_paths)):
        itkimg = itk.imread(data_paths[i])
        depth,zoom,offsetX,offsetY = get_depth_and_zoom_C52(itkimg)
        resampled = linearize_image(itkimg,depth,zoom,offsetY,True)
        print(depth, zoom)
        itkresampled = itk.GetImageFromArray(resampled.astype(np.float32))
        filename = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])
        itk.imwrite(itkresampled, filename)
        
def convert_boxes_to_linear(pathname):
    data_paths = sorted(glob(pathname+"-Annotations/*f?????.mha"))
    for i in range(len(data_paths)):
        bxfile = pathname+"-Annotations/"+os.path.basename(data_paths[i])[:-4]+".boxes.json"
        print(bxfile)

        itkimg = itk.imread(data_paths[i], itk.UC)
        depth,zoom,offsetX,offsetY = get_depth_and_zoom_C52(itkimg)

        itkboximg_linear = linearize_image(itkimg,depth,zoom,offsetY)
        itkboximg_linear.FillBuffer(0)
        itkimg.FillBuffer(0)
        boximg = itk.GetArrayViewFromImage(itkimg)

        with open(bxfile) as f:
          data = json.load(f)
        slices = data['boxes']
        num_box_slices = len(slices)
        box_count = 0
        for s in range(num_box_slices):
            boxes = slices[s]['boxes']
            num_boxes = len(boxes)
            for b in range(num_boxes):
                box_indx = boxes[b]['indices']
                print("   ",box_indx)
                start = [int(item) for item in box_indx[0]]
                end = [int(item) for item in box_indx[1]]
                if(start[0]>end[0]):
                    start[0], end[0] = end[0], start[0]
                if(start[1]>end[1]):
                    start[1], end[1] = end[1], start[1]
                boximg[start[1]:end[1],start[0]:end[0]] = box_count+1
                box_count += 1
        itkboximg_linear = linearize_image(itkimg,depth,zoom,offsetY,False)  #use False to not interpolate
        bxfile = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])[:-4]+".boxes.mha"
        itk.imwrite(itkboximg_linear, bxfile)
        labelmap = itk.GetArrayFromImage(itkboximg_linear).astype(int)
        props = skimage.measure.regionprops(labelmap)
        if box_count != len(props):
            print("ERROR: # of linearized boxes (",len(props),") != number of boxes (",box_count,")")
        box_count = 0
        for s in range(num_box_slices):
            boxes = slices[s]['boxes']
            num_boxes = len(boxes)
            for b in range(num_boxes):
                bbox = props[box_count].bbox
                boxes[b]['indices'] = [[bbox[1],bbox[0],0],[bbox[3],bbox[2],0]]
                box_count += 1
        bxfile = pathname+"-Annotations-Linear/"+os.path.basename(data_paths[i])[:-4]+".boxes.json"
        with open(bxfile, 'w') as outfile:
            json.dump(data, outfile)


In [10]:
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
#pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
pathname = "../../Data/TestingData/BAMC-PTXSliding"
convert_video_to_linear(pathname)

Using non-standard frame rate 10240/512


0 ../../Data/TestingData/BAMC-PTXSliding\image_3925135436261_clean.mov
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627


Using non-standard frame rate 10240/512


1 ../../Data/TestingData/BAMC-PTXSliding\image_3929217595322_clean.mov
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627


In [6]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
#pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
#pathname = "../../Data/TestingData/BAMC-PTXSliding"
convert_slices_to_linear(pathname)

12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
12 1.0
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 

12 1.2627627627627627
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048
   Resampling with zoom = 0.8048048048048048
12 0.8048048048048048


In [12]:
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
#pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
pathname = "../../Data/TestingData/BAMC-PTXSliding"
convert_annotations_to_linear(pathname)

   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.2627627627627627
   Resampling with zoom = 1.2627627627627627
12 1.262762762762

In [11]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXNoSliding"
#pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
#pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
#pathname = "../../Data/TestingData/BAMC-PTXSliding"

convert_boxes_to_linear(pathname)